In [29]:
import cv2
from pathlib import Path
import numpy as np
import pandas as pd

In [15]:
PATH = Path('../VOCdevkit/VOC2007/JPEGImages')
NEW_PATH = Path('../VOCdevkit/VOC2007/JPEGImages_300_300')

In [22]:
str(list(PATH.iterdir())[0]).split('/')[-1]

'007622.jpg'

In [23]:
for path in list(PATH.iterdir()):
    img = cv2.imread(str(path))
    resized = cv2.resize(img, (300, 300))
    fname = str(path).split('/')[-1]
    cv2.imwrite(str(NEW_PATH/fname), resized)

In [36]:
def make_bb_px(y, x_shape):
    """Makes an image of size x retangular bounding box"""
    r, c = x_shape
    Y = np.zeros((r, c))
#     print(y)
    y = hw_bb(y).astype(np.int)
    Y[y[0]:y[2], y[1]:y[3]] = 1.
    return Y


def to_bb(Y):
    """Convert mask Y to a bounding box, assumes 0 as background nonzero object"""
    cols, rows = np.nonzero(Y)
    if len(cols)==0: return np.zeros(4, dtype=np.float32)
    top_row = np.min(rows)
    left_col = np.min(cols)
    bottom_row = np.max(rows)
    right_col = np.max(cols)
    return np.array([left_col, top_row, right_col, bottom_row], dtype=np.float32)


def hw_bb(bb): 
    """Transform from width-height format to bounding box format.
    
    width-height: [X, Y, width, height]
    bounding-box: [Y, X, left-bottom, right-top]
    """
    return np.array([bb[1], bb[0], bb[3]+bb[1]-1, bb[2]+bb[0]-1])


def bb_hw(bb): 
    """Transform from bounding box format to width-height format.
    
    width-height: [X, Y, width, height]
    bounding-box: [Y, X, left-bottom, right-top]
    """
    return np.array([bb[1], bb[0], bb[3]-bb[1]+1, bb[2]-bb[0]+1])

In [47]:
# resize bounding boxes accordingly
def resize_bbox(x_shape, bbox):
    Y = make_bb_px(bbox, x_shape)
    y_resize = cv2.resize(Y, x_shape)
    y = bb_hw(to_bb(y_resize))
    return y

In [45]:
PATH = Path('../VOCdevkit/VOC2007/')
train_bbox_df = pd.read_csv(PATH/'tmp/train_bbox.csv')
val_bbox_df = pd.read_csv(PATH/'tmp/val_bbox.csv')

In [50]:
y_300 = []
for bbox in train_bbox_df['bbox']:
    y = np.array([int(b) for b in bbox.split()])
    y_300.append(resize_bbox((300, 300), y))

In [51]:
train_bbox_df['bbox_300'] = y_300

In [53]:
train_bbox_df

,fn,label,bbox,bbox_300
0,VOCdevkit/VOC2007/JPEGImages/000012.jpg,6,155 96 196 174,"[155.0, 96.0, 145.0, 173.0]"
1,VOCdevkit/VOC2007/JPEGImages/000017.jpg,12,89 77 314 259,"[89.0, 77.0, 211.0, 223.0]"
2,VOCdevkit/VOC2007/JPEGImages/000023.jpg,14,2 1 241 461,"[2.0, 1.0, 240.0, 299.0]"
3,VOCdevkit/VOC2007/JPEGImages/000026.jpg,6,89 124 248 88,"[89.0, 124.0, 211.0, 87.0]"
4,VOCdevkit/VOC2007/JPEGImages/000032.jpg,0,103 77 272 106,"[103.0, 77.0, 197.0, 105.0]"
5,VOCdevkit/VOC2007/JPEGImages/000033.jpg,0,8 106 491 157,"[8.0, 106.0, 292.0, 156.0]"
6,VOCdevkit/VOC2007/JPEGImages/000034.jpg,18,115 166 245 234,"[115.0, 166.0, 185.0, 134.0]"
7,VOCdevkit/VOC2007/JPEGImages/000035.jpg,14,217 97 248 221,"[217.0, 97.0, 83.0, 203.0]"
8,VOCdevkit/VOC2007/JPEGImages/000036.jpg,11,26 78 293 266,"[26.0, 78.0, 274.0, 222.0]"
9,VOCdevkit/VOC2007/JPEGImages/000042.jpg,18,262 31 238 264,"[262.0, 31.0, 38.0, 263.0]"
